In [3]:
from pathlib import Path
import requests

def download_one_file_of_raw_data(year: int, month: int) -> Path:
    """
    """
    URL = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
    response = requests.get(URL)
    
    if response.status_code == 200:
        path = f'../data/raw/rides_{year}-{month:02d}.parquet'
        open(path, "wb").write(response.content)
        return path
    else:
        raise Exception(f"{URL} is not available")

In [4]:
download_one_file_of_raw_data(year=2022, month=1)

'../data/raw/rides_2022-01.parquet'

In [11]:
import pandas as pd

rides = pd.read_parquet('../data/raw/rides_2022-01.parquet')

# lots of columns we only need two of them which are tpep_pickup_datetime (when? time) and PULocationID (where? place)
rides = rides[['tpep_pickup_datetime', 'PULocationID']].rename(columns={'tpep_pickup_datetime': 'pickup_datetime', 'PULocationID': 'pickup_location_id'})

rides.head(20)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
5,2022-01-01 00:40:15,138
6,2022-01-01 00:20:50,233
7,2022-01-01 00:13:04,238
8,2022-01-01 00:30:02,166
9,2022-01-01 00:48:52,236


In [13]:
# validate the data
rides['pickup_datetime'].describe()

# there are some rows in the file that shouldn't be there
rides = rides[rides.pickup_datetime >= '2022-01-01']
rides = rides[rides.pickup_datetime < '2022-02-01']

In [16]:
rides['pickup_datetime'].describe()

count                       2463879
mean     2022-01-17 01:58:40.393674
min             2022-01-01 00:00:08
25%             2022-01-09 15:37:56
50%             2022-01-17 12:11:54
75%             2022-01-24 13:49:37
max             2022-01-31 23:59:58
Name: pickup_datetime, dtype: object

In [19]:
rides.to_parquet('../data/transformed/validated_rides_2022-01.parquet')